# Hierarchical equality

Reproducing the hierarchical equality experiment in [Finding Alignments Between Interpretable Causal Variables and Distributed Neural Representations](https://arxiv.org/pdf/2303.02536.pdf).

In [13]:
FORCE_CPU = False
SEED = 2384

HIDDEN_SIZE = 16
TASK_INPUT_SIZE = 1
TASK_TRAIN_SIZE = 2**22
TASK_TEST_SIZE = 2**12
TASK_BATCH_SIZE = 2048
TASK_LR = 1e-3
TASK_EPOCHS = 100

SAVE_MODEL = True
LOAD_MODEL = False
MODEL_PATH = "saved_models/hierarchical-equality.pt"
MODEL_TRAIN_DETAILS_PATH = "saved_models/hierarchical-equality-train-details.pickle"

## Setup

In [2]:
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/SamAdamDay/mechanistic-interpretability-projects.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

import plotly.io as pio
pio.renderers.default = "colab+vscode"

Running as a Jupyter notebook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_4970/1821744192.py:13: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_4970/1821744192.py:14: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [14]:
from typing import Tuple
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
from numpy.typing import NDArray

from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [4]:
np.random.seed(SEED)
torch.manual_seed(SEED)

In [5]:
if not FORCE_CPU and torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

cuda


## Train a model

In [6]:
# create a three layer MLP in torch
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size, bias=True)
        self.layer2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.layer3 = nn.Linear(hidden_size, output_size, bias=True)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.softmax(self.layer3(x))
        return x

The dataset is divided equally between positive and negative classes. The positive instances are equally divided between:
- (A, A, B, B)
- (A, B, C, D)

The negative instances are equally divided between:
- (A, A, B, C)
- (A, B, C, C)

(There are more options but I chose to omit them)

In [7]:
def generate_data(size: int, input_size: int) -> Tuple[NDArray, NDArray]:

    X =  np.empty((size, input_size, 4))
    y = np.empty((size,))

    group_size = size // 4
    # (A, A, B, B)
    X[:group_size, :, 0] = X[:group_size, :, 1] = np.random.standard_normal((group_size, TASK_INPUT_SIZE))
    X[:group_size, :, 2] = X[:group_size, :, 3] = np.random.standard_normal((group_size, TASK_INPUT_SIZE))
    # (A, B, C, D)
    X[group_size:2*group_size, :, :] = np.random.standard_normal((group_size, TASK_INPUT_SIZE, 4))
    # (A, A, B, C)
    X[2*group_size:3*group_size, :, 0] = X[2*group_size:3*group_size, :, 1] = np.random.standard_normal((group_size, TASK_INPUT_SIZE))
    X[2*group_size:3*group_size, :, 2:4] = np.random.standard_normal((group_size, TASK_INPUT_SIZE, 2))
    # (A, B, C, C)
    X[3*group_size:, :, 0:2] = np.random.standard_normal((group_size, TASK_INPUT_SIZE, 2))
    X[3*group_size:, :, 2] = X[3*group_size:, :, 3] = np.random.standard_normal((group_size, TASK_INPUT_SIZE))

    y[:size // 2] = 1
    y[size // 2:] = 0

    return X, y

In [8]:
train_x, train_y = generate_data(TASK_TRAIN_SIZE, TASK_INPUT_SIZE)
test_x, test_y = generate_data(TASK_TEST_SIZE, TASK_INPUT_SIZE)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(4194304, 1, 4)
(4194304,)
(4096, 1, 4)
(4096,)


In [9]:
train_dataset = TensorDataset(torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float())
train_dataloader = DataLoader(train_dataset, batch_size=TASK_BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float())
test_dataloader = DataLoader(test_dataset, batch_size=TASK_BATCH_SIZE, shuffle=True)

In [10]:
model = MLP(TASK_INPUT_SIZE * 4, HIDDEN_SIZE, 2)
model.to(device)

print(model)

MLP(
  (layer1): Linear(in_features=4, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=16, bias=True)
  (layer3): Linear(in_features=16, out_features=2, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)


In [11]:
if LOAD_MODEL:

    model.load_state_dict(torch.load(MODEL_PATH))

    with open(MODEL_TRAIN_DETAILS_PATH, "rb") as f:
        dict = pickle.load(f)
        train_losses = dict["train_losses"]
        test_losses = dict["test_losses"]
        test_accuracies = dict["test_accuracies"]

else:

    optimizer = optim.SGD(model.parameters(), lr=TASK_LR)
    loss_fn = nn.CrossEntropyLoss()

    train_losses = np.empty((TASK_EPOCHS,))
    test_losses = np.empty((TASK_EPOCHS,))
    test_accuracies = np.empty((TASK_EPOCHS,))

    for epoch in range(TASK_EPOCHS):
        total_train_loss = 0.0
        for batch in tqdm(train_dataloader, desc=f"Training [{epoch+1}/{TASK_EPOCHS}]"):
            batch_x, batch_y = batch
            batch_x = batch_x.flatten(start_dim=1)
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            output = model(batch_x)
            loss = loss_fn(output, batch_y.long())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_dataloader)
        train_losses[epoch] = avg_train_loss

        with torch.no_grad():

            total_test_loss = 0.0
            total_test_accuracy = 0.0
            
            for batch in test_dataloader:
                batch_x, batch_y = batch
                batch_x = batch_x.flatten(start_dim=1)
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)

                output = model(batch_x)
                loss = loss_fn(output, batch_y.long())

                pred = torch.argmax(output, dim=1)
                total_test_accuracy += torch.sum(pred == batch_y.long()).item() / len(pred)

                total_test_loss += loss.item()

            avg_test_loss = total_test_loss / len(test_dataloader)
            test_losses[epoch] = avg_test_loss

            avg_test_accuracy = total_test_accuracy / len(test_dataloader)
            test_accuracies[epoch] = avg_test_accuracy

        print(f"Epoch [{epoch+1}/{TASK_EPOCHS}] train loss: {avg_train_loss:0.5f}, test acc: {avg_test_accuracy:0.5%}")

Training [1/100]:   0%|          | 0/2048 [00:00<?, ?it/s]

Training [1/100]: 100%|██████████| 2048/2048 [00:25<00:00, 81.69it/s] 


Epoch [0/100] train loss: 0.69513, test acc: 49.92676%


Training [2/100]: 100%|██████████| 2048/2048 [00:24<00:00, 84.56it/s] 


Epoch [1/100] train loss: 0.69414, test acc: 49.92676%


...

Epoch [97/100] train loss: 0.65954, test acc: 67.57812%


Training [99/100]: 100%|██████████| 2048/2048 [00:23<00:00, 87.45it/s] 


Epoch [98/100] train loss: 0.65809, test acc: 67.91992%


Training [100/100]: 100%|██████████| 2048/2048 [00:23<00:00, 88.18it/s] 

Epoch [99/100] train loss: 0.65662, test acc: 67.96875%


In [15]:
if SAVE_MODEL:
    torch.save(model.state_dict(), MODEL_PATH)
    with open(MODEL_TRAIN_DETAILS_PATH, "wb") as f:
        pickle.dump({"train_losses": train_losses, "test_losses": test_losses, "test_accuracies": test_accuracies}, f)

In [24]:
def plot_loss(train_losses, test_losses):
    epochs = list(range(1, len(train_losses) + 1))

    # Create a line plot for train loss
    train_trace = go.Scatter(
        x=epochs,
        y=train_losses,
        mode='lines',
        name='Train Loss'
    )

    # Create a line plot for test loss
    test_trace = go.Scatter(
        x=epochs,
        y=test_losses,
        mode='lines',
        name='Test Loss'
    )

    # Create the layout for the plot
    layout = go.Layout(
        title='Train and Test Loss over Epochs',
        xaxis=dict(title='Epoch'),
        yaxis=dict(title='Loss'),
        showlegend=True
    )

    # Combine the train and test traces into a data list
    data = [train_trace, test_trace]

    # Create the figure and display the plot
    fig = go.Figure(data=data, layout=layout)
    fig.show()

plot_loss(train_losses, test_losses)

In [25]:
fig = px.line(x=range(1, len(test_accuracies) + 1), y=test_accuracies, title="Test Accuracy over Epochs")
fig.update_layout(xaxis_title="Epoch", yaxis_title="Accuracy", yaxis_tickformat="0.0%")

In [26]:
with torch.no_grad():

    total_train_loss = 0.0
    total_train_accuracy = 0.0
    
    for batch in tqdm(train_dataloader):
        batch_x, batch_y = batch
        batch_x = batch_x.flatten(start_dim=1)
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        output = model(batch_x)

        pred = torch.argmax(output, dim=1)
        total_train_accuracy += torch.sum(pred == batch_y.long()).item() / len(pred)

    avg_train_accuracy = total_train_accuracy / len(train_dataloader)

print(f"Train accuracy: {avg_train_accuracy:0.5%}")

100%|██████████| 2048/2048 [00:20<00:00, 99.07it/s] 

Train accuracy: 68.41300%
